In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
!pip install requests
!pip install tabulate
!pip install "colorama>=0.3.8"
!pip install future

In [ ]:
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

In [ ]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

In [ ]:
%%time
path = "/kaggle/input/talkingdata-adtracking-fraud-detection/train.csv"
s = !wc -l {path}

print(s)

In [ ]:
df_tmp = pd.read_csv(path, nrows=5)
df_tmp.head()

In [ ]:
df_tmp.info()

In [ ]:
chunksize = 100000
from tqdm import tqdm

In [ ]:
df1 = pd.read_csv(path, chunksize = chunksize, usecols = ['is_attributed'] )
df1 = pd.DataFrame(df1.get_chunk(1))
print(type(df1))
#print(len(ch1))
##import dask.dataframe as dd
##path = "/kaggle/input/talkingdata-adtracking-fraud-detection/train.csv"
##df = dd.read_csv(path)

In [ ]:
print(type(df1.info()))

In [ ]:
%%time
df_list = [] # list to hold the batch dataframe
i=0
for df_chunk in tqdm(pd.read_csv(path, chunksize = chunksize, usecols = ['is_attributed'] )):
     
    # Neat trick from https://www.kaggle.com/btyuhas/bayesian-optimization-with-xgboost
    # Using parse_dates would be much slower!
    #df_chunk['pickup_datetime'] = df_chunk['pickup_datetime'].str.slice(0, 16)
    #df_chunk['pickup_datetime'] = pd.to_datetime(df_chunk['pickup_datetime'], utc=True, format='%Y-%m-%d %H:%M')
    
    # Can process each chunk of dataframe here
    # clean_data(), feature_engineer(),fit()
    df1 = pd.DataFrame(df_chunk)
    if i<10:
        print(df1.shape)
    # Alternatively, append the chunk to list and merge all
    df_list.append(df_chunk) 
    i += 1

In [ ]:
print(len(df_list))
print(df_list[0][1500:1600])

In [ ]:
from sys import getsizeof
getsizeof(df_list)
type(df_list[0])
#получаем в чанке 0 элемент с индексом 1504 (строка) и колонкой 0 ( у нас одна колонка)
df_list[0].iloc[1504,0]

In [ ]:
s=0
for i in range(len(df_list)):
    s += df_list[i].sum()

#во всех чанках у нас столько единиц
print(s)

In [ ]:
# а всего записей у нас у нас:
df_list[-1]

In [ ]:
# то есть 184903889+1
#итого: доля единиц:
print(456846/184903890*100) #в процентах доля единиц среди всех данных 0.247%

In [ ]:
# Merge all dataframes into one dataframe
train_df = pd.concat(df_list)

# Delete the dataframe list to release memory
del df_list

# See what we have loaded
train_df.info()

In [ ]:
#from tqdm import tqdm
#from progressbar import progressbar as pb

#!ls -l /kaggle/input/talkingdata-adtracking-fraud-detection

# Load data into H2O
path = "/kaggle/input/talkingdata-adtracking-fraud-detection/train.csv"
#i = 0

chunks = []
for chunk in tqdm(pd.read_csv(path, chunksize=1000)):
    #i +=1
    #print(i)
    chunks.append(chunk)
    
#df = pd.read_csv(path)
#df.info()

In [ ]:
#возьмем временно другой датасет train_sample
# Load data into H2O
path = "/kaggle/input/talkingdata-adtracking-fraud-detection/train_sample.csv"
df = pd.read_csv(path)
df.info()

In [ ]:
#не заполнены колонки со временем: нужно их заполнить
#выведем заполненные
df_samp = df[pd.notnull(df['attributed_time'])]
df_samp

In [ ]:
type(df_samp.iloc[0,6])

In [ ]:
df.iloc[:10]

In [ ]:
#надо заполнить колонку attributed_time так:
#для тех строк у которых известно attributed_time вычисляем среднюю разницу между click_time и attributed_time
#это и будет добавка к колонке click_time, далее просто заполняем незаполненные путем добавления времени к click_time

In [ ]:
df.drop('attributed_time', axis = 1, inplace=True)
df.info()

In [ ]:
labelEncoder = LabelEncoder()
for col in df.columns:
    df[col] = labelEncoder.fit_transform(df[col])

df = h2o.H2OFrame(df)
df = df.asfactor()

In [ ]:
df.describe ()

смущает что время было преобразовано в большой набор чисел, может ли это повлиять на результаты автообучения???

Ниже в коде: 
могу ли я не расщеплять исходный датасет на тестовый и тренировочный? (скорее всего да)

In [ ]:
#настройка и  тренировка

train = df

y = "is_attributed"
x_train = train.columns
x_train.remove(y)

aml = H2OAutoML(max_runtime_secs=300, seed = 1)
aml.train(x = x_train, y = y, training_frame = train)

In [ ]:
#просмотреть результаты работы
lb = aml.leaderboard
lb.head()

In [ ]:
#preds = aml.predict(test)
aml.training_info

In [ ]:
#тестовые данные
path = "/kaggle/input/talkingdata-adtracking-fraud-detection/test.csv"
df_test = pd.read_csv(path)
df_test.info()

In [ ]:
df_test[:10]

In [ ]:
#дропаем лишнюю колонку, чтобы была структура такая же как и у train но без target колонки
df_test.drop('click_id', axis = 1, inplace=True)
df_test.info()

In [ ]:
labelEncoder = LabelEncoder()

for col in df_test.columns:
    df_test[col] = labelEncoder.fit_transform(df_test[col])

df_test = h2o.H2OFrame(df_test)
df_test = df_test.asfactor()

In [ ]:
preds = aml.predict(df_test)
preds

In [ ]:
type(preds)

In [ ]:
print(len(preds))

In [ ]:
df_preds = h2o.as_list(preds)

In [ ]:
sum(df_preds['predict']==1)

In [ ]:
#то есть для стольких элементов 97634 из 18790469 предсказана 1, для остальных 0 по лучшей модели
#доля:
print(97634/18790469*100) # в процентах, то есть больше чем исходно единиц в основном тренировочном 
#датасете (было 0.24707214109989792% )